<a href="https://colab.research.google.com/github/Alafiade/ML-/blob/main/TITANIC_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING THE DEPENCIES

In [32]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

UPLOADING FILES MY  LOCAL DISK TO GOOGLE COLAB ENVIRONMENT

In [3]:
from google.colab import files
uploaded= files.upload()

Saving titanic.csv to titanic.csv


In [4]:
# Reading the CSV file Titanic into a pandas DataFrame
titanic= pd.read_csv('titanic.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
# Checking number of rows of the data frame
titanic.shape

(418, 12)

In [6]:
# Getting some information about the dataset
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [14]:
#Finding null values in the dataset
titanic.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

DATA PREPROCESSING

In [13]:
# Filling in the null values in Age columns
titanic['Age']= titanic['Age'].fillna(titanic['Age'].median())
# Filling in the null values in Fare
titanic['Fare']= titanic['Fare'].fillna(titanic['Fare'].median())
# Convertimng the non-numerical values in Cabin column to NaN
titanic['Cabin']= pd.to_numeric(titanic['Cabin'],errors='coerce')
titanic['Cabin'] = titanic['Cabin'].fillna(0)


In [15]:
# Encoding Sex column
titanic['Sex']= titanic['Sex'].replace({'female':1,'male':0})

In [23]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,0.0,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,0.0,S
2,894,0,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,0.0,Q
3,895,0,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,0.0,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,0.0,S


SPLITTING THE DATASET INTO X anf Y

In [27]:
X = titanic.drop(['Name','Fare','Embarked','Ticket'],axis=1)
Y = titanic['Fare']

In [28]:
#Printing the values of X and Y
print(X)
print(Y)

     PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch  Cabin
0            892         0       3    0  34.5      0      0    0.0
1            893         1       3    1  47.0      1      0    0.0
2            894         0       2    0  62.0      0      0    0.0
3            895         0       3    0  27.0      0      0    0.0
4            896         1       3    1  22.0      1      1    0.0
..           ...       ...     ...  ...   ...    ...    ...    ...
413         1305         0       3    0  27.0      0      0    0.0
414         1306         1       1    1  39.0      0      0    0.0
415         1307         0       3    0  38.5      0      0    0.0
416         1308         0       3    0  27.0      0      0    0.0
417         1309         0       3    0  27.0      1      1    0.0

[418 rows x 8 columns]
0        7.8292
1        7.0000
2        9.6875
3        8.6625
4       12.2875
         ...   
413      8.0500
414    108.9000
415      7.2500
416      8.0500
417     22.3

STANDARDIZING THE DATA

In [30]:
#Importing the StandardScaler
scaler = StandardScaler()
#Fit the scaler to thhe data and transform the features
X_standardized = scaler.fit_transform(X)

In [31]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1,random_state=2)

XGBOOST

In [34]:
# Create an instance of XGBRegressor
xgb_model = xgb.XGBRegressor()
# Fit the model on the training data
xgb_model.fit(X_train,Y_train)
# Make predictions on the test data
Y_pred =  xgb_model.predict(X_test)
# Evaluate the model's performance
mse = mean_squared_error(Y_test,Y_pred)
# Print Mean Squared Error
print('Mean Squared Error:',mse)


Mean Squared Error: 2323.1170717602267


TUNING HYPERPARAMETERS TO IMPROVE THE MODEL'S PERFORMANCE

In [40]:
# Using GridSearchCV to find the best combination of hyperparameters
# Define the hyperparameter grid
param_grid = {
    'n_estimators':[100,200,300],
    'max_depth':[3,4,5],
    'learning_rate':[0.01,0.1,0.2],
    'subsample':[0.5,0.8,1],
    'colsample_bytree':[0.5,0.8,1]
}
# Create an instance of XGBRegressor
xgb_model = xgb.XGBRegressor()
# Create the grid search object
grid_search = GridSearchCV(estimator=xgb_model,param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# Fit the gris search object to the data
grid_search.fit(X_train,Y_train)
# Get the best combination of hyperparameters
best_params = grid_search.best_params_
print('Best hyperparameters:', best_params)
# Train the model with the best hyperparameters
best_xgb_model = xgb.XGBRegressor(**best_params)
best_xgb_model.fit(X_train,Y_train)
# Make predictions on the test data
Y_pred = best_xgb_model.predict(X_test)
#  Evaluate the model's performance
mse = mean_squared_error(Y_test,Y_pred)
print('Mean Squarred Error:',mse)

Best hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300, 'subsample': 1}
Mean Squarred Error: 1779.5093487437546


CONTROLLING THE COMPLEXITY OF THE MODEL

In [37]:
# Fit the model with L1 and L2 regularization(alpha and lambda)
xgb_model = xgb.XGBRegressor(reg_alpha=1, reg_lambda=1)
xgb_model.fit(X_train,Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)